# Testing in notebooks

Some useful references:

- [Using Pytest in Jupyter](https://medium.com/@mefengl/using-pytest-in-jupyter-notebooks-a-practical-guide-1ba8e02af288)
- [The ipytest module](https://github.com/chmp/ipytest)

The `ipytest` module lets us run tests _inside_ a notebook, which enables the gradual evolution of code from living in a notebook during interactive exploration, and then being moved to modules and packages as we modularize and abstract reusable parts, all while remaining tested.

In a notebook that we want to run tests for, we start by importing and configuring `ipytest` (the docs have more details on the various options):

In [8]:
import ipytest
ipytest.autoconfig()

If we have a collection of tests in a single cell, we can use the `%%ipytest` cell magic, provided by the package. Note that flag to ignore an annoying `DeprecationWarning`, it kicks in only the 2nd time we run the cell but helps keep the visual noise down.

In [11]:
%%ipytest -qq -W ignore::DeprecationWarning

# This test will pass
def test_addition():
    assert 1 + 1 == 2

# This one will fail
def test_problem():
    assert 1 + 1 == 3

# And we may want to have a test we don't run for now...
import pytest
@pytest.mark.skip(reason="no way of currently testing this")
def test_mystery():
    pass

.Fs                                                                                          [100%]
============================================= FAILURES =============================================
___________________________________________ test_problem ___________________________________________

    def test_problem():
>       assert 1 + 1 == 3
E       assert (1 + 1) == 3

/var/folders/j1/n8kn9ftd7257n2rvkkzlj3mc0010dw/T/ipykernel_39407/3734627355.py:7: AssertionError
===================================== short test summary info ======================================
FAILED t_de6dfc9099544e969c1912f0c3674ff3.py::test_problem - assert (1 + 1) == 3


The beauty of `ipytest` is that it can find tests defined _anywhere_ in the notebook and run them:

In [12]:
ipytest.run('-W ignore::DeprecationWarning')

.Fs                                                                                          [100%]
============================================= FAILURES =============================================
___________________________________________ test_problem ___________________________________________

    def test_problem():
>       assert 1 + 1 == 3
E       assert (1 + 1) == 3

/var/folders/j1/n8kn9ftd7257n2rvkkzlj3mc0010dw/T/ipykernel_39407/3734627355.py:7: AssertionError
===================================== short test summary info ======================================
FAILED t_de6dfc9099544e969c1912f0c3674ff3.py::test_problem - assert (1 + 1) == 3
1 failed, 1 passed, 1 skipped in 0.01s


<ExitCode.TESTS_FAILED: 1>

:::{note}
Strictly speaking, `ipytest` finds tests _in the kernel's namespace_, not in the notebook file. This means that if you restert the kernel, you need to re-run the cells defining tests for them to be found. 
:::

If we define more tests now:

In [13]:
def test_bad():
    assert 1 == 0

def test_crash():
    1/0

They will also be found:

In [14]:
ipytest.run('-W ignore::DeprecationWarning')

.FsFF                                                                                        [100%]
============================================= FAILURES =============================================
___________________________________________ test_problem ___________________________________________

    def test_problem():
>       assert 1 + 1 == 3
E       assert (1 + 1) == 3

/var/folders/j1/n8kn9ftd7257n2rvkkzlj3mc0010dw/T/ipykernel_39407/3734627355.py:7: AssertionError
_____________________________________________ test_bad _____________________________________________

    def test_bad():
>       assert 1 == 0
E       assert 1 == 0

/var/folders/j1/n8kn9ftd7257n2rvkkzlj3mc0010dw/T/ipykernel_39407/714497975.py:2: AssertionError
____________________________________________ test_crash ____________________________________________

    def test_crash():
>       1/0
E       ZeroDivisionError: division by zero

/var/folders/j1/n8kn9ftd7257n2rvkkzlj3mc0010dw/T/ipykernel_39407/714497975.py

<ExitCode.TESTS_FAILED: 1>